# Translacja
Chodzi o to żeby przekształcić tekst, aby dało się go wyrenderować przy pomocy mojego specjalnego fontu dla systemu stenografii.
Założeniem systemu stenografii jest to, żeby nie trzeba było odrywać pióra przy pisaniu wyrazu.

Idea systemu:
1. Pisze się wokół linii środkowej ciągnąc kreskę do linii górnej lub dolnej i prawie zawsze (oprócz końca wyrazu) wracając do linii środkowej
1. Kreski mają różne (proste) kształy (w sumie 11)
1. Spółgłoski oznaczane są kreską od linii środkowej w górę lub w dół (różne spółgłoski dla lustrzanej kreski idącej w górę i w dół)
1. Samogłoski oznaczane są kreską od linii górnej lub dolnej do środkowej (ta sama samogłoska może być napisana wracając z góry jak i z dołu po przez lustrzane odbicie)
1. Jeśli występują po sobie dwie spółgłoski lub dwie samogłoski, to między nimi następuje powrót przy pomocy linii pionowej
1. Na końcu spółgłoski może dodany być jeden z dwóch akcentów (kreska i kółko/pętelka)
1. TODO: Na końcu samogłoski może dodany być akcent (mała skośna linia i powrót) który zmienia `e` na `ę`, `a` na `ą`, `i` na `ie`
1. TODO: Można pisać też samogłoski poziomo, na linii środkowej, co jest przydatne gdy samogłoska następuje po samogłosce

## Transkrypcja

Pierwszą fazą jest podzielenie tekstu na głoski.

In [1]:
import re

clean_pattern = re.compile(r'(\W|\d|_)+')
def cleanup(t):
    return re.sub(clean_pattern, ' ', t.lower())+' '

cleanup("Nie! 11  dąsać się-pROszę")

'nie dąsać się proszę '

In [2]:
split_pattern = re.compile(r'(sz|cz|rz|dzi|dz|dź|dż|ch|si|ci|ni)')
def dziel(t):
    pos = 0
    for m in re.finditer(split_pattern, t):
        s = m.span()
        if m.start() != pos:
            for c in t[pos:m.start()]:
                yield c
        yield m[1]
        pos = m.end()
    if pos < len(t):
        for c in t[pos:]:
            yield c

print(list(dziel('zaszczute rzeźnickie dżdżownice dźwięczą dzisiaj dzdz ')))

['z', 'a', 'sz', 'cz', 'u', 't', 'e', ' ', 'rz', 'e', 'ź', 'ni', 'c', 'k', 'i', 'e', ' ', 'dż', 'dż', 'o', 'w', 'ni', 'c', 'e', ' ', 'dź', 'w', 'i', 'ę', 'cz', 'ą', ' ', 'dzi', 'si', 'a', 'j', ' ', 'dz', 'dz', ' ']


Z `ś` + `si`, `ć` + `ci` oraz `ń` + `ni` nie jest oczywiste której użyć. Jeśli po niej jest samogłoska, to chciłoby się uzyć `ś` a jeśli spółgłoska, to lepiej osobno `s` `i` (ze względu na komplikacje przy powtórzonych samogłoskach).
Podobną regułę moza przyjąć dla `dź` + `dzi`.

In [3]:
def czy_samlgloska(s):
    return s in ['a','i','e','o','y','u','ę','ą','ó']

def czy_spolgloska(s):
    return str.isalpha(s) and not czy_samlgloska(s)

In [4]:
def popraw(iter):
    prev = next(iter)
    for g in iter:
        if prev in ['si', 'ci', 'ni', 'dzi'] and czy_spolgloska(g):
            yield prev[:-1]
            yield 'i'
        else:
            yield prev
        prev = g
    yield prev

print(list(popraw(dziel('sieć i sito dzisiaj dziadek ciepło ciska'))))

['si', 'e', 'ć', ' ', 'i', ' ', 's', 'i', 't', 'o', ' ', 'dz', 'i', 'si', 'a', 'j', ' ', 'dzi', 'a', 'd', 'e', 'k', ' ', 'ci', 'e', 'p', 'ł', 'o', ' ', 'c', 'i', 's', 'k', 'a']



Utożsamiane są niektóre głoski. Mamy podstawienia:
- `ch` -> `h`
- `rz` -> `ż`
- `dzi` -> `dź`
- `rz` -> `ż`
- `ó` -> `u`
- `si` -> `ś`
- `ci` -> `ć`
- `ni` -> `ń`

Akcent pierwszy (pętelka) daje podstawienia (`1` oznacza górną formę akcentu, `3` oznacza dolną)
- `ś` -> `s1`
- `ć` -> `c1`
- `dź` -> `d3`
- `ź` -> `z3`
- `ń` -> `n3`

Akcent drugi (kreska w górę/dół oznaczana przez `2`/`4`) daje:
- `sz` -> `s2`
- `cz` -> `c2`
- `dż` -> `d4`
- `ż` -> `z4`

In [5]:
podstawienia = {
    'ch': 'h',
    'rz': 'z4',
    'dzi': 'd3',
    'rz': 'z4',
    'ó': 'u',
    'si': 's1',
    'ci': 'c1',
    'ni': 'n3',
    'ś': 's1',
    'ć': 'c1',
    'dź': 'd3',
    'ź': 'z3',
    'ń': 'n3',
    'sz': 's2',
    'cz': 'c2',
    'dż': 'd4',
    'ż': 'z4'
}

def podstaw(iter):
    for c in iter:
        if c in podstawienia:
            yield podstawienia[c]
        else:
            yield c

Spółgłoski dzielą się na górne i dolne.

In [6]:
gorne = {
    'm',
    's',
    't',
    'l',
    'p',
    'k',
    'w',
    'r',
    'c',
    'dz',
    'x'
}

dolne = {
    'n',
    'z',
    'd',
    'ł',
    'b',
    'g',
    'f',
    'j',
    'h',
    'v',
    'q'
}

Po górnej spółgłosce może występować tylko górna samogłoska (oznaczona upper-case), a po dolnej - dolna.

Jeśli po spółgłosce jest spółgłoska, to między nimi musi znaleźć się linia wracająca do środkowej linii (górna `\` lub dolna `/`).

Tymczasowo podobnie dla samogłosek, póki nie są obsługiwane poziome formy samogłosek.

In [33]:
akc_pattern = re.compile(r'[1234]$')

def bez_akcentu(c):
    return re.sub(akc_pattern, '', c)

def polacz(iter):
    prev = next(iter)
    prev_sam = czy_samlgloska(prev)
    prev_gorna = not prev_sam and bez_akcentu(prev) in gorne
    next_up = True
    for c in iter:
        sam = czy_samlgloska(c)
        gorna = not sam and bez_akcentu(prev) in gorne
        go_up = next_up
        next_up = False
        if sam and prev_sam:
            # strange case, go up to solve it
            prev = prev + '\\'
            next_up = True
        elif sam and not prev_sam and prev != ' ':
            if prev_gorna:
                next_up = True
        elif not sam and not prev_sam and prev != ' ':
            if prev_gorna:
                prev = prev + '\\'
            else:
                prev = prev + '/'
        # emit before going to next
        if prev_sam and go_up:
            yield prev.upper()
        else:
            yield prev
        prev = c
        prev_sam = sam
        prev_gorna = gorna
    # emit last one
    if prev_sam and next_up:
        yield prev.upper()
    else:
        yield prev
        
print(list(polacz(['s','t','a','d','n','o','e'].__iter__())))

['s', '\\', 't', 'A', 'd', '/', 'n', 'o', '\\', 'E']


Wygląd akcentu `ą` i `ę` zależy od tego czy jest to samogłoska górna czy dolne. Dlatego dopiero teraz możemy o nim zdecydować.

TODO

In [39]:
def konwertuj(s):
    return polacz(podstaw(popraw(dziel(cleanup(s)))))

print(''.join(konwertuj('i tak i dzisiejszy dzień zaczął się bardzo nietypowo')))

I tak/ i dzis1ej/s2y d3en3/ zac2ął/ s1ę bar/dzO n3etypowo 
